In [3]:
import sys
from osgeo import gdal

class ValidateCloudOptimizedGeoTIFFException(Exception):
    pass

In [6]:
if int(gdal.VersionInfo('VERSION_NUM')) < 2020000:
    print("GDAL 2.2 or above required, you have {}".format(gdal.VersionInfo()))

GDAL 2.2 or above required, you have 2010300


In [42]:
filename = '/home/andrew/Data/usgs/ot_test.tif'

In [43]:
gdal.PushErrorHandler()
ds = gdal.Open(filename)
gdal.PopErrorHandler()
if ds is None:
    print("Invalid file : %s" % gdal.GetLastErrorMsg())

In [44]:
if ds.GetDriver().ShortName != 'GTiff':
    print("The file is not a GeoTIFF")

In [45]:
main_band = ds.GetRasterBand(1)
ovr_count = main_band.GetOverviewCount()
if filename + '.ovr' in ds.GetFileList():
    print("Overviews should be internal")

In [46]:
block_size = main_band.GetBlockSize()
if block_size[0] == main_band.XSize:
    print("Full resolution image is not tiled")

In [47]:
if main_band.XSize >= 512 or main_band.YSize >= 512:
    if ovr_count == 0:
        print("The file should have overviews")

In [51]:
ds.GetMetadata_List()

['AREA_OR_POINT=Area', 'Band_1=band 1 surface reflectance']

In [48]:
ifd_offset = [int(main_band.GetMetadataItem('IFD_OFFSET', 'TIFF'))]
if ifd_offset[0] != 8:
    print("The offset of the main IFD should be 8. It is %d instead" % ifd_offset[0])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [54]:
for i in range(ovr_count):
    # Check that overviews are by descending sizes
    ovr_band = ds.GetRasterBand(1).GetOverview(i)
    if i == 0:
        if ovr_band.XSize > main_band.XSize or ovr_band.YSize > main_band.YSize:
                print("First overview has larger dimension than main band")
    else:
        prev_ovr_band = ds.GetRasterBand(1).GetOverview(i-1)
        if ovr_band.XSize > prev_ovr_band.XSize or ovr_band.YSize > prev_ovr_band.YSize:
                print("Overview of index %d has larger dimension than overview of index %d" % (i, i-1))

    block_size = ovr_band.GetBlockSize()
    if block_size[0] == ovr_band.XSize:
        print("Overview of index %d is not tiled" % i)

In [55]:
for i in range(ovr_count):
    # Check that the IFD of descending overviews are sorted by increasing
    # offsets
    ifd_offset.append(int(ovr_band.GetMetadataItem('IFD_OFFSET', 'TIFF')))
    if ifd_offset[-1] < ifd_offset[-2]:
        if i == 0:
            print(
                "The offset of the IFD for overview of index %d is %d, "
                "whereas it should be greater than the one of the main "
                "image, which is at byte %d" %
                (i, ifd_offset[-1], ifd_offset[-2]))
        else:
            print(
                "The offset of the IFD for overview of index %d is %d, "
                "whereas it should be greater than the one of index %d, "
                "which is at byte %d" %
                (i, ifd_offset[-1], i-1, ifd_offset[-2]))

NameError: name 'ifd_offset' is not defined

In [21]:
# Check that the imagery starts by the smallest overview and ends with
# the main resolution dataset
data_offset = [int(main_band.GetMetadataItem('BLOCK_OFFSET_0_0', 'TIFF'))]
for i in range(ovr_count):
    ovr_band = ds.GetRasterBand(1).GetOverview(i)
    data_offset.append(int(
        ovr_band.GetMetadataItem('BLOCK_OFFSET_0_0', 'TIFF')))
if data_offset[-1] < ifd_offset[-1]:
    if ovr_count > 0:
        print(
            "The offset of the first block of the smallest overview "
            "should be after its IFD")
    else:
        print(
            "The offset of the first block of the image should "
            "be after its IFD")

NameError: name 'ifd_offset' is not defined

In [56]:
for i in range(len(data_offset)-2, 0, -1):
    if data_offset[i] < data_offset[i+1]:
        print(
            "The offset of the first block of overview of index %d should "
            "be after the one of the overview of index %d" %
            (i-1, i))
if len(data_offset) >= 2 and data_offset[0] < data_offset[1]:
    print(
        "The offset of the first block of the main resolution image "
        "should be after the one of the overview of index %d" %
        (ovr_count - 1))

In [24]:
!cp /home/andrew/Data/usgs/LT050430271984116-SC20170101123907/LT05_L1TP_043027_19840425_20161004_01_T1_sr_band1.tif /home/andrew/Data/usgs/otest.tif

In [29]:
!gdalinfo ~/Data/usgs/otest.tif

Driver: GTiff/GeoTIFF
Files: /home/andrew/Data/usgs/otest.tif
Size is 5000, 5000
Coordinate System is:
PROJCS["Albers",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.2572221010042,
                AUTHORITY["EPSG","7019"]],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Albers_Conic_Equal_Area"],
    PARAMETER["standard_parallel_1",29.5],
    PARAMETER["standard_parallel_2",45.5],
    PARAMETER["latitude_of_center",23],
    PARAMETER["longitude_of_center",-96],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]]]
Origin = (-1815585.000000000000000,3014805.000000000000000)
Pixel Size = (30.000000000000000,-30.000000000000000)
Metadata:
  AREA_OR_POINT=Area
  Band_1=band 1 surface reflectance
Image Structure Metadata:
  INTERLEAVE=BAND
Corner 

In [32]:
!gdalinfo ~/Data/usgs/otest.tif

Driver: GTiff/GeoTIFF
Files: /home/andrew/Data/usgs/otest.tif
Size is 5000, 5000
Coordinate System is:
PROJCS["Albers",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.2572221010042,
                AUTHORITY["EPSG","7019"]],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Albers_Conic_Equal_Area"],
    PARAMETER["standard_parallel_1",29.5],
    PARAMETER["standard_parallel_2",45.5],
    PARAMETER["latitude_of_center",23],
    PARAMETER["longitude_of_center",-96],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]]]
Origin = (-1815585.000000000000000,3014805.000000000000000)
Pixel Size = (30.000000000000000,-30.000000000000000)
Metadata:
  AREA_OR_POINT=Area
  Band_1=band 1 surface reflectance
Image Structure Metadata:
  INTERLEAVE=BAND
Corner 

In [23]:
!gdaladdo --help

Usage: gdaladdo [-r {nearest,average,gauss,cubic,cubicspline,lanczos,average_mp,average_magphase,mode}]
                [-ro] [-clean] [-q] [-oo NAME=VALUE]* [--help-general] filename levels

  -r : choice of resampling method (default: nearest)
  -ro : open the dataset in read-only mode, in order to generate
        external overview (for GeoTIFF datasets especially)
  -clean : remove all overviews
  -q : turn off progress display
  -b : band to create overview (if not set overviews will be created for all bands)
  filename: The file to build overviews for (or whose overviews must be removed).
  levels: A list of integral overview levels to build. Ignored with -clean option.

Useful configuration variables :
  --config USE_RRD YES : Use Erdas Imagine format (.aux) as overview format.
Below, only for external overviews in GeoTIFF format:
  --config COMPRESS_OVERVIEW {JPEG,LZW,PACKBITS,DEFLATE} : TIFF compression
  --config PHOTOMETRIC_OVERVIEW {RGB,YCBCR,...} : TIFF photometric interp.

In [31]:
!gdaladdo -r cubic ~/Data/usgs/otest.tif 2 4 8 

0...10...20...30...40...50...60...70...80...90...100 - done.


In [40]:
!gdal_translate ~/Data/usgs/otest.tif ~/Data/usgs/ot_test.tif -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=DEFLATE

Input file size is 5000, 5000
0...10...20...30...40...50...60...70...80...90...100 - done.


In [41]:
!gdalinfo ~/Data/usgs/ot_test.tif

Driver: GTiff/GeoTIFF
Files: /home/andrew/Data/usgs/ot_test.tif
Size is 5000, 5000
Coordinate System is:
PROJCS["Albers",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.2572221010042,
                AUTHORITY["EPSG","7019"]],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Albers_Conic_Equal_Area"],
    PARAMETER["standard_parallel_1",29.5],
    PARAMETER["standard_parallel_2",45.5],
    PARAMETER["latitude_of_center",23],
    PARAMETER["longitude_of_center",-96],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]]]
Origin = (-1815585.000000000000000,3014805.000000000000000)
Pixel Size = (30.000000000000000,-30.000000000000000)
Metadata:
  AREA_OR_POINT=Area
  Band_1=band 1 surface reflectance
Image Structure Metadata:
  COMPRESSION=DEFLATE
 

In [57]:
import urllib

In [58]:
urllib.parse.uses_relative.append('s3')